# Generate Density Map

In [1]:
import numpy as np
import scipy
from scipy import spatial
from scipy import ndimage
from tqdm import tqdm

In [2]:
images=np.load('images.npy')
images_gt=np.load('images_gt.npy',allow_pickle=True)

In [3]:
def generate_density(X,Y):
    point=np.array([X,Y]).T
    leafsize=2048
    tree=spatial.KDTree(point.copy(),leafsize=leafsize)
    distances,locations=tree.query(point,k=4)
    density=np.zeros([480,640],dtype=np.float32)
    for i,pt in enumerate(point):
        pt2d=np.zeros([480,640],dtype=np.float32)
        pt=np.round(pt).astype('int')
        pt2d[pt[1],pt[0]]=1000
        sigma=min(max(1,int(np.sum(distances[i][1:3])*0.1)),8)
        density+=ndimage.filters.gaussian_filter(pt2d, sigma, mode='constant')
    return density

In [ ]:
images_density=np.zeros([2000,480,640],dtype=np.float32)
for i in tqdm(range(2000)):
    images_density[i]=generate_density(images_gt[i][:,0],images_gt[i][:,1])

 93%|████████████████████████████████████████████████████████████████████████▏     | 1852/2000 [12:13<01:14,  1.98it/s]

In [ ]:
np.save('images_density.npy',images_density)